In [1]:
import pandas as pd

df = pd.read_csv('data/colors_2.csv', delimiter=':')
print(df)


     x1   x2   x3
0   255   25   25
1   255   64   25
2   255  102   25
3   255  140   25
4   255  179   25
..  ...  ...  ...
56  230    0  153
57  230    0  153
58  230    0   76
59  230    0   38
60    0    0    0

[61 rows x 3 columns]


In [2]:
df_bw = pd.DataFrame(df)

for idx in df.index:
    brightness = int(0.299 * df.loc[idx][0] + 0.587 * df.loc[idx][1] + 0.114 * df.loc[idx][2])
    df_bw.loc[idx] = [brightness, brightness, brightness]

# раскомментировать для ч/б режима
df = pd.DataFrame(df_bw)

print(df)
print(df_bw)


     x1   x2   x3
0    93   93   93
1   116  116  116
2   138  138  138
3   161  161  161
4   184  184  184
..  ...  ...  ...
56   86   86   86
57   86   86   86
58   77   77   77
59   73   73   73
60    0    0    0

[61 rows x 3 columns]
     x1   x2   x3
0    93   93   93
1   116  116  116
2   138  138  138
3   161  161  161
4   184  184  184
..  ...  ...  ...
56   86   86   86
57   86   86   86
58   77   77   77
59   73   73   73
60    0    0    0

[61 rows x 3 columns]


In [3]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, init='k-means++', n_init='auto')
y_pred = kmeans.fit_predict(df)
df['label'] = y_pred

print(kmeans.cluster_centers_)
print(df)

# df.to_csv('output/labeled.csv', index=False)


[[ 87.86956522  87.86956522  87.86956522]
 [203.66666667 203.66666667 203.66666667]
 [ 38.66666667  38.66666667  38.66666667]
 [158.11764706 158.11764706 158.11764706]]
     x1   x2   x3  label
0    93   93   93      0
1   116  116  116      0
2   138  138  138      3
3   161  161  161      3
4   184  184  184      1
..  ...  ...  ...    ...
56   86   86   86      0
57   86   86   86      0
58   77   77   77      0
59   73   73   73      0
60    0    0    0      2

[61 rows x 4 columns]


In [4]:
centroids = pd.DataFrame()
centroids[['x1', 'x2', 'x3']] = kmeans.cluster_centers_
centroids['sum'] = centroids.sum(axis=1)
centroids.sort_values(by=['sum'], inplace=True)

with open("output/index.html", "w") as html_file:
    html_file.write(
        '<html><head><style>table {border-collapse: collapse; border: none} td {border: none; padding: 1.5vw}</style></head><body><table>')
    for label in centroids.index:
        cluster = pd.DataFrame(df.iloc[df.index[df.label == label].tolist()])
        cluster['sum'] = cluster[['x1', 'x2', 'x3']].sum(axis=1)
        cluster.sort_values(by=['sum'], inplace=True)

        html_file.write(
            f'<tr><td style="background-color:rgb({centroids.loc[label][0]}, {centroids.loc[label][1]}, {centroids.loc[label][2]})">{label}</td>')

        for idx in cluster.index:
            html_file.write(
                f'<td style="background-color:rgb({cluster.loc[idx][0]}, {cluster.loc[idx][1]}, {cluster.loc[idx][2]})"></td>')

    html_file.write("</table></body></html>")
